# Why DCGAN instead of GAN?

Generative Adversarial Networks (GANs) use two networks to generate data (generator) and classify if it is real or generated (discriminator). These two networks continuously tries to improve their abilities and after training one can supply some noise to the generator and it will generate some output, e.g. a cat if it has been trained on cat pictures.  

Problems with GANs are that they are hard to train, the model can collapse, slight changes in variables/architecture can destabilize the model, i.e. training GANs is somewhat of an art according to some. Deep Convolutional GANs (DCGANs) solves/mitigates these problems.  

### Main differences between DCGANs and GANs
Making three changes to GANs allows them to become more stable and produce better results.  

The first change is to replace pooling functions with convolution layers.  
The second change is to remove fully connected layers on top of convolutional layers.  
The third change to use batch normalization (although with some caveats...)

![title](./guidelines.png)

Source: DCGAN paper by Radford, Metz & Chintala

## General Architecture of DCGANs


## Challenges with DCGANs

In [8]:
# Imports
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# to make this notebook's output stable across runs (from lab2)
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 1.12.0


In [13]:
class DCGAN(object):
    
    def __init__(self):
        """
        Constructs DCCGAN with tf.Keras

        """
        
        # Set parameters
        self.rows = 28
        self.columns = 28
        self.channels = 1
        self.shape = (self.rows, self.columns, self.channels) # Image shape 
        self.batch_size = 128
        
        # Build the models
        self.generator = self._buildGenerator()
        self.discriminator = self._buildDiscriminator()
    
        # Compile the models
        self.generator.compile()
        self.discriminator.compile()
        
        # Train the models
        
        
    # Currently following this: https://julianzaidi.wordpress.com/2017/04/24/deep-convolution-gan-dcgan-architecture-and-training/
    def _buildGenerator(self):
        """
        Constructs the generator part of the DCGAN
        """
        ##########
        # Layers #
        ##########
        # 1. Input
        # 2. Reshape layer
        # 3. Transposed conv layer
        # 4. Transposed conv layer
        # 5. Transposed conv layer
        # 6. Transposed conv layer
        # 7. Transposed conv layer
        
        #model = tf.keras.Input() # Sequential or Functional API?
        # In the blog post they use a reshape, is that neccessary? Investigate
        noise = np.random.normal(loc=0 , scale=0.2 , shape=(self.batch_size, self.rows, 1, 1))
        model = tf.keras.Sequential()
        model.add(layers.Conv2DTranspose())
        model.add(layers.Conv2DTranspose())
        model.add(layers.Conv2DTranspose())
        model.add(layers.Conv2DTranspose())
        model.add(layers.Conv2DTranspose())
    
    def _buildDiscriminator(self):
        """
        Constructs the discriminator part of the DCGAN
        """
        ##########
        # Layers #
        ##########
        # 1. Input
        # 2. Conv layer
        # 3. Conv layer
        # 4. Conv layer
        # 5. Conv layer
        # 6. Dense layer
        
        model = tf.keras.Sequential()
        model.add(layers.Conv2D())
        model.add(layers.Conv2D())
        model.add(layers.Conv2D())
        model.add(layers.Conv2D())
        model.add(layers.Dense())
    
    def trainModel(self):
        """
        Trains the generator and discriminator
        """
        


In [11]:
#noise = np.random.normal(loc=0 , scale=0.2 , size=(3, 2, 1, 1))
#print(noise)

[[[[ 0.13897881]]

  [[ 0.11966697]]]


 [[[-0.47545968]]

  [[ 0.2212227 ]]]


 [[[ 0.19881932]]

  [[-0.03660763]]]]
